In [1]:
# Bootstrap: ensure project root is importable from any notebook location
import sys, os
from pathlib import Path
d
PROJECT_ROOT = Path(os.environ.get("HOME")) / "iwtc-lab"
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# If lib.paths exists, optionally pin CWD to project root and ensure tree
try:
    from lib.paths import ROOT, ensure_tree, set_cwd_to_root  # optional module
    set_cwd_to_root()
    ensure_tree()
    print("Using lib.paths; ROOT:", ROOT)
except Exception as e:
    print("lib.paths not available; using HOME-based root:", PROJECT_ROOT)

NameError: name 'd' is not defined

## Contents
- [lib.dice](#libdice)
- [Environment checks](#envchecks)

## <a id="libdice"></a>lib.dice
### Quick examples

In [ ]:
from lib.dice import roll, roll_adv, roll_dis
print( roll("3d6+2").detail )
print( roll("4d6kh3").detail )
print( roll_adv(+5).detail )
print( roll_dis(-1).detail )

### Minimal tests
The goal here is just to catch obvious regressions in parse semantics.

In [ ]:
from lib.dice import parse
tests = {
    "d20": (1,20,None,None,0),
    "3d6+2": (3,6,None,None,2),
    "4d6kh3": (4,6,"kh",3,0),
    "4d6dl1-1": (4,6,"dl",1,-1),
}
for expr, expected in tests.items():
    got = parse(expr)
    assert got == expected, (expr, got, expected)
print("✅ lib.dice parse tests: OK")

## <a id="envchecks"></a>Environment checks
Useful when something fails above.

In [ ]:
import sys, os, platform
print("Python:", sys.version.split()[0])
print("Executable:", sys.executable)
print("Platform:", platform.platform())
print("PYTHONPATH:", os.environ.get("PYTHONPATH"))
# Confirm project root discoverable
print("Project root in sys.path?", str((Path(os.environ['HOME'])/'iwtc-lab')) in sys.path)

# IWTC Tools Demonstration

This notebook serves as a functional showcase for IWTC-Lab core libraries.  
Each section demonstrates one capability, integrating smoothly with the others.

---

## Table of Contents

| # | Section | Key Topics |
|:-:|:--|:--|
| 0 | [**Bootstrap — Environment Setup**](#bootstrap--environment-setup) |  |
| 1 | [**Dice Roller — `lib.dice`**](#dice-roller--libdice) | [Basic roll examples](#basic-roll-examples), [Advantage & disadvantage tests](#advantage-and-disadvantage-tests) |
| 2 | [**SRD Data Demonstration — `lib.srd_reader`**](#srd-data-demonstration--libsrd_reader) | [Load and normalize monster](#load-and-normalize-monster) |
| 3 | [**Rolling Demonstration — `lib.roll_adapter`**](#rolling-demonstration--libroll_adapter) | [Strength save](#strength-saving-throw-example), [Attack roll](#attack-roll-example-to-hit-and-damage) |
| 4 | [**Environment Summary**](#environment-summary) |  |


## <a id="bootstrap--environment-setup"></a>0. Bootstrap — Environment Setup

**Purpose:**  
Prepare the Jupyter environment for running code in this notebook.

This step:
- Ensures paths and imports are correctly configured.
- Activates or verifies the IWTC virtual environment.
- Confirms access to all core libraries (`lib.dice`, `lib.srd_reader`, `lib.roll_adapter`, etc.).
- Prints a brief summary of project root and data directories.

Run this section once per session before using any tools.


In [2]:
# === IWTC Bootstrap (fixed) ===
import sys, os
from pathlib import Path
import importlib.util

print("=== IWTC Bootstrap ===")

# Project root: ENV override → default ~/iwtc-lab
PROJECT_ROOT = Path(os.environ.get("IWTC_ROOT", Path.home() / "iwtc-lab")).resolve()
os.environ["IWTC_ROOT"] = str(PROJECT_ROOT)

LIB_DIR = PROJECT_ROOT / "lib"

# 1) Ensure the PARENT of 'lib' is on sys.path (the project root)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
    print(f"📚 Added project root to sys.path: {PROJECT_ROOT}")
else:
    print(f"📚 Project root already on sys.path: {PROJECT_ROOT}")

# 2) Quick existence checks
print(f"📂 lib exists: {'✅' if LIB_DIR.exists() else '⛔'}  ({LIB_DIR})")

# 3) Sanity: show which 'lib' Python would import
spec = importlib.util.find_spec("lib")
print("🔎 find_spec('lib'):", "FOUND" if spec else "NOT FOUND")

# 4) Try imports
try:
    import lib.dice as dice
    print("✅ import lib.dice OK")
except Exception as e:
    print("⛔ import lib.dice failed:", repr(e))

try:
    import lib.srd_reader as srd_reader
    print("✅ import lib.srd_reader OK")
except Exception as e:
    print("⚠️ import lib.srd_reader:", repr(e))

try:
    import lib.roll_adapter as roll_adapter
    print("✅ import lib.roll_adapter OK")
except Exception as e:
    print("⚠️ import lib.roll_adapter:", repr(e))

print("========================")


=== IWTC Bootstrap ===
📚 Added project root to sys.path: /Users/charissophia/iwtc-lab
📂 lib exists: ✅  (/Users/charissophia/iwtc-lab/lib)
🔎 find_spec('lib'): FOUND
✅ import lib.dice OK
✅ import lib.srd_reader OK
✅ import lib.roll_adapter OK


## <a id="dice-roller--libdice"></a>1. Dice Roller — `lib.dice`

**Purpose:**  
Demonstrate the core dice-rolling utilities that power other IWTC-Lab modules.

These tests ensure that any system calling `lib.dice` (such as SRD or homebrew tools) can depend on consistent roll behavior.


### <a id="basic-roll-examples"></a>Basic roll examples
Simple tests of standard dice expressions.


In [ ]:
from lib.dice import roll

# Basic examples
exprs = ["1d20", "3d6+2", "4d6kh3", "2d8+1d4+3", "4d6kh3+2"]
for e in exprs:
    r = roll(e)
    print(f"{e:<10s} → {r.detail} = {r.total}")


### <a id="advantage-and-disadvantage-tests"></a>Advantage and disadvantage tests
Verify that the higher or lower of two rolls is correctly selected.


In [ ]:
from lib.dice import roll_adv, roll_dis

# Advantage / disadvantage examples
expr = "1d20+5"
print(f"Rolling {expr} with advantage:")
adv = roll_adv(expr)
print("Result:", adv.detail, "=", adv.total)

print(f"\nRolling {expr} with disadvantage:")
dis = roll_dis(expr)
print("Result:", dis.detail, "=", dis.total)


## <a id="srd-data-demonstration--libsrd_reader"></a>2. SRD Data Demonstration — `lib.srd_reader`

**Purpose:**  
Demonstrate that IWTC-Lab can read and normalize a single monster record from the imported SRD JSON files.

This section:
- Locates the appropriate SRD monsters file (preferring 2024 > 2014).  
- Extracts a monster by name.  
- Normalizes its data into the minimal `statblock.v1` shape used for rolling tests.  
- Prints a concise summary showing provenance and key statistics.


### <a id="load-and-normalize-monster"></a>Load and normalize monster

In [ ]:
from lib.srd_reader import load_monster_raw, normalize_minimal
from pathlib import Path
import json

MONSTER_NAME = "Goblin"

# Load the monster (function returns just the record)
monster_raw = load_monster_raw(MONSTER_NAME)

if monster_raw is None:
    raise ValueError(f"❌ Monster '{MONSTER_NAME}' not found in SRD data.")

# Normalize to minimal internal shape
monster = normalize_minimal(monster_raw)
monster["_source_file"] = "data/srd/2014/5e-SRD-Monsters.json"  # optional manual provenance

# Display key stats
print(f"Name: {monster['name']}")
print(f"Abilities: {monster['abilities']}")
print(f"Proficiency Bonus: +{monster['proficiency_bonus']}")
saves = monster.get('saving_throws', {})
print(f"Saving Throws: {', '.join(saves) if saves else '(none)'}")

# Inspect all attack-style actions for the Goblin
if "actions" not in monster_raw:
    print("No actions found in raw SRD entry.")
else:
    print(f"Actions for {monster_raw.get('name', '(unknown)')}:")
    for a in monster_raw["actions"]:
        name = a.get("name", "(unnamed)")
        desc = a.get("desc", "").strip().replace("\n", " ")
        print(f"- {name}: {desc}")


## <a id="rolling-demonstration--libroll_adapter"></a> 3. Rolling Demonstration — `lib.roll_adapter`

**Purpose:**  
Use the normalized SRD monster to run example rolls via the dice library:
- Strength saving throw
- Attack (to-hit + damage)

**Notes:**
- `roll_strength_save()` prefers an explicit STR save bonus if present; otherwise it uses the STR ability modifier.
- `pick_attack()` falls back to computing a to-hit bonus if the SRD entry doesn’t include `attack_bonus`.
- `roll_attack()` rolls to-hit once and then each damage expression listed.

If you haven’t run Section 2 in this session, the first cell below will safely reload the creature.


### <a id="strength-saving-throw-example"></a>Strength saving throw example


In [ ]:
# Ensure we have a normalized monster available
try:
    monster
except NameError:
    from lib.srd_reader import load_monster_raw, normalize_minimal
    MONSTER_NAME = "Goblin"
    monster_raw = load_monster_raw(MONSTER_NAME)
    monster = normalize_minimal(monster_raw)

from lib.roll_adapter import roll_strength_save

sv = roll_strength_save(monster)
print(f"Monster: {monster['name']}")
print(f"Expr:    {sv['expr']}  ({sv['modifier_detail']})")
print("Detail: ", sv["result"].detail)
print("Total:  ", sv["result"].total)


### <a id="attack-roll-example-to-hit--and-damage"></a>Attack roll example (to-hit and damage)


In [6]:
# Select a specific attack ("Shortbow") from the raw SRD, normalize, and roll
import re
from typing import Dict, Any, List
from lib.dice import roll  # used by roll_attack via lib.roll_adapter
from lib.roll_adapter import roll_attack, ability_mod
from lib.srd_reader import normalize_minimal

def _normalize_damage_list(dmg) -> List[Dict[str, Any]]:
    """Accepts ['1d6+2', ...] or [{'expr':'1d6+2','type':'slashing'}, ...] -> list of dicts."""
    out: List[Dict[str, Any]] = []
    if not isinstance(dmg, list):
        return out
    for item in dmg:
        if isinstance(item, str):
            out.append({"expr": item, "type": ""})
        elif isinstance(item, dict):
            expr = item.get("expr") or item.get("damage_dice") or item.get("dice") or item.get("damage_roll") or ""
            dtype = item.get("type") or item.get("damage_type") or ""
            if expr:
                out.append({"expr": expr, "type": dtype})
    return out

def extract_attack_by_name(raw: Dict[str, Any], name: str) -> Dict[str, Any]:
    """Find an action by exact name (case-insensitive) and return {'name','to_hit','damage':[{expr,type},...]}."""
    target = name.lower()
    for a in raw.get("actions", []) or []:
        if a.get("name", "").lower() != target:
            continue

        # To-hit: prefer explicit attack_bonus; else parse from desc; else compute fallback.
        to_hit = a.get("attack_bonus")
        if to_hit is None and isinstance(a.get("desc"), str):
            m = re.search(r"\+(\d+)\s*to hit", a["desc"])
            if m:
                to_hit = int(m.group(1))
        if to_hit is None:
            abil = normalize_minimal(raw)["abilities"]
            to_hit = max(ability_mod(abil["str"]), ability_mod(abil["dex"])) + 2  # conservative PB fallback

        # Damage components
        dmg: List[Dict[str, Any]] = []
        if isinstance(a.get("damage"), list):
            dmg = _normalize_damage_list(a["damage"])
        if not dmg and isinstance(a.get("desc"), str):
            # Last-ditch parse from text like "Hit: 5 (1d6 + 2) piercing"
            for m in re.finditer(r"(\d+d\d+(?:\s*[+\-]\s*\d+)?)\s*(?:\((\w+)\))?", a["desc"]):
                dice = m.group(1).replace(" ", "")
                dtype = (m.group(2) or "").lower()
                dmg.append({"expr": dice, "type": dtype})

        return {"name": a.get("name", "Attack"), "to_hit": int(to_hit), "damage": _normalize_damage_list(dmg)}

    raise ValueError(f"Attack '{name}' not found in SRD actions.")

# Ensure we have monster_raw/monster in scope (reload if needed)
try:
    monster_raw
except NameError:
    from lib.srd_reader import load_monster_raw
    MONSTER_NAME = "Goblin"
    monster_raw = load_monster_raw(MONSTER_NAME)
monster = normalize_minimal(monster_raw)

# Choose the Shortbow specifically
atk = extract_attack_by_name(monster_raw, "Shortbow")

print(f"Monster: {monster['name']}")
print(f"Attack:  {atk['name']}  |  To-hit bonus: +{atk['to_hit']}")

def pretty_type(t) -> str:
    if isinstance(t, dict):
        return (t.get("name") or t.get("index") or "").strip()
    return (str(t) if t else "").strip()

# Preview damage (robust to strings/dicts)
dlist = _normalize_damage_list(atk.get("damage", []))
if not dlist:
    print("Damage:  (none parsed from SRD entry)")
else:
    parts = []
    for d in dlist:
        expr = d.get("expr")
        if isinstance(expr, dict):
            expr = expr.get("expr") or expr.get("damage_dice") or str(expr)
        dtype = pretty_type(d.get("type", ""))
        parts.append(f"{expr} {dtype}".strip())
    print("Damage:", ", ".join(parts))

# Roll output
res = roll_attack({"name": atk["name"], "to_hit": atk["to_hit"], "damage": dlist})

print("\nTo-hit roll:")
print(" ", res["to_hit_expr"], "=>", res["to_hit"].detail, "=", res["to_hit"].total)

if res["damage"]:
    print("\nDamage rolls:")
    for i, d in enumerate(res["damage"], 1):
        dtype = pretty_type(d.get("type", ""))
        dtype_txt = f" ({dtype.capitalize()})" if dtype else ""
        print(f"  {i}. {d['expr']} => {d['result'].detail} = {d['result'].total}{dtype_txt}")
else:
    print("\n(No damage components found.)")


Monster: Goblin
Attack:  Shortbow  |  To-hit bonus: +4
Damage: 1d6+2 Piercing

To-hit roll:
  1d20+4 => 1d20[8]+4 = 12

Damage rolls:
  1. 1d6+2 => 1d6[1]+2 = 3 (Piercing)
